### IBM - Data Science - Capstone Project Notebook

In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install folium

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium 
from IPython.display import display_html

### Part 1 - parse Toronto Neighbourhood and post code wiki

In [3]:
# Load Toronto Neighbourghoods and postal codes wiki
page = requests.get("http://zims-en.kiwix.campusafrica.gos.orange.com/wikipedia_en_all_nopic/A/List_of_postal_codes_of_Canada:_M")

In [24]:
# Parse and Convert into dataframe
soup = BeautifulSoup(page.content, 'lxml')
tab = str(soup.table)
dfs = pd.read_html(tab)


In [25]:
# Clean up naming as per assignment suggestions
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df=dfs[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [26]:
# Drop records with not assigned boroughs
df = df[df.Borough != 'Not assigned']

#Rename columns to match assignment
df = df.rename(columns={'Postcode':'Postal Code'},inplace=False)

In [27]:
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [28]:
df.shape

(211, 3)

### Part 2 - enrich neighbourhoods with locations

In [29]:
geoloc = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
geoloc.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [30]:
df_with_loc = pd.merge(df,geoloc,on='Postal Code')
df_with_loc.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


###  Part 3 - cluster the neighbourhoods

In [31]:
# Cluster using the k means algorithm

k=5
locations_only = df_with_loc.drop(['Postal Code','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(locations_only)
kmeans.labels_
df_with_clusters = df_with_loc
df_with_clusters.insert(0, 'Cluster Labels', kmeans.labels_)

In [32]:
# plot on map
# The latitude of Toronto, ON, Canada is 43.651070, and the longitude is -79.347015.
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=9)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
for lat, lon, neighbourhood, cluster in zip(df_with_loc['Latitude'], df_with_loc['Longitude'], df_with_loc['Neighbourhood'], df_with_loc['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters